<a href="https://colab.research.google.com/github/thinkdeepai/spark-training/blob/main/Spark_Training_WordCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download and install dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar xf spark-3.2.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyngrok
!hostname

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
e3f3665a5d4b


In [ ]:
import os

# set the environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: ignored

In [ ]:
# verify the availability of Spark v3.2.4
spark

In [ ]:
# For this you need to create a free account on ngrok.com and enter your auth token below (replace AUHT_TOKEN)
NGROK_AUTH_TOKEN = "2CXOq3cZSdyxDgL97aR8zRo69KF_2PiUKLMyzpcxqq5TuLYQf"

# Export Spark history URL
from pyngrok import ngrok, conf

conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(4050, "http").public_url

print(f"Link to Spark History: {connection_string}")

Link to Spark History: https://abb7-34-125-23-129.ngrok-free.app


In [ ]:
# Read the input file and Calculating words count
# Try the following text (Caesar's Gallic Wars): https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/gallic.mb.txt
!wget https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/gallic.mb.txt
text_file = sc.textFile("gallic.mb.txt")

--2023-07-07 12:44:35--  https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/gallic.mb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 496659 (485K) [text/plain]
Saving to: ‘gallic.mb.txt’

gallic.mb.txt       100%[===================>] 485.02K  --.-KB/s    in 0.04s   

2023-07-07 12:44:35 (11.2 MB/s) - ‘gallic.mb.txt’ saved [496659/496659]



In [ ]:
# Ex1: Implement the Wordcount
# 1.1 Split each line per word
text_words = text_file.flatMap(lambda x: x.split(" "))
text_words_filtered = text_words.filter(lambda x: x != '')
# 1.2 Create tuple for each word
text_tuples = text_words_filtered.map(lambda x: (x,1))
# 1.2 For the same word, add each tuple add the result
text_counts = text_tuples.reduceByKey(lambda x,y: x+y)

In [ ]:
# Ex2: Display the result
text_counts.take(10)

[('BOOK', 8),
 ('1', 10),
 ('Chapter', 402),
 ('Gaul', 65),
 ('is', 216),
 ('into', 250),
 ('three', 68),
 ('of', 3192),
 ('Belgae', 18),
 ('in', 1412)]

In [ ]:
# Ex3: Sort RDD by key (word) ascending
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sortBy.html#pyspark.RDD.sortBy
text_counts.sortByKey().take(100)

[('"Dumnorix', 1),
 ('"For', 1),
 ('"I', 1),
 ('"If', 2),
 ('"In', 2),
 ('"Leap,', 1),
 ('"Prevail,"', 1),
 ('"Since', 1),
 ('"Since,', 1),
 ('"Soldiers,', 2),
 ('"That', 15),
 ('"That,', 1),
 ('"The', 1),
 ('"These', 1),
 ('"Victory,"', 1),
 ('"Why', 3),
 ('"any,', 1),
 ('"brethren"', 1),
 ('"cippi."', 1),
 ('"friend"', 1),
 ('"he', 5),
 ('"if', 3),
 ('"in', 1),
 ('"kinsmen,"', 1),
 ('"leaving', 1),
 ('"neither', 1),
 ('"no', 1),
 ('"nor', 1),
 ('"not', 3),
 ('"on', 1),
 ('"particularly,', 1),
 ('"remember', 1),
 ('"since', 1),
 ('"that', 21),
 ('"the', 3),
 ('"they', 5),
 ('"to', 3),
 ('"trust', 1),
 ('"what', 2),
 ('"who', 2),
 ('"why', 1),
 ('"with', 2),
 ('"you', 2),
 ("'brethren'", 1),
 ("'brothers'", 1),
 ("'king", 1),
 ('(Gaul,', 1),
 ('(Now', 1),
 ('(Roman)', 1),
 ('(a', 2),
 ('(affrighted),', 1),
 ('(already', 1),
 ('(among', 1),
 ('(an', 1),
 ('(as', 8),
 ('(at', 1),
 ('(because', 1),
 ('(by', 2),
 ('(except', 1),
 ('(fearing', 1),
 ('(for', 15),
 ('(forasmuch', 1),
 ('(from

In [ ]:
# Ex4: Sort RDD by value (count) descending
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sortBy.html#pyspark.RDD.sortBy
text_words_sorted = text_counts.sortBy(lambda x: x[1], ascending=False)

In [ ]:
text_words_parititoned = text_words_sorted.repartition(10)

In [ ]:
# Ex5: Write RDD to disk
text_words_parititoned.saveAsTextFile("words_partitions/output.txt")

In [ ]:
text_words_persisted = text_words_sorted.persist()

In [ ]:
text_words_persisted.take(1000)

[('the', 6956),
 ('of', 3192),
 ('and', 3079),
 ('to', 2771),
 ('that', 1485),
 ('in', 1412),
 ('he', 1345),
 ('a', 1273),
 ('they', 1137),
 ('their', 1105),
 ('had', 1039),
 ('by', 1002),
 ('his', 845),
 ('from', 772),
 ('was', 756),
 ('with', 711),
 ('were', 635),
 ('on', 597),
 ('be', 590),
 ('which', 589),
 ('as', 577),
 ('for', 546),
 ('not', 506),
 ('all', 474),
 ('them', 425),
 ('Chapter', 402),
 ('at', 371),
 ('should', 318),
 ('this', 310),
 ('our', 309),
 ('been', 307),
 ('having', 301),
 ('it', 293),
 ('who', 290),
 ('great', 290),
 ('being', 288),
 ('him', 251),
 ('into', 250),
 ('Caesar', 250),
 ('those', 245),
 ('would', 232),
 ('these', 227),
 ('when', 226),
 ('one', 225),
 ('could', 223),
 ('The', 220),
 ('is', 216),
 ('are', 210),
 ('an', 208),
 ('have', 202),
 ('men', 188),
 ('enemy', 186),
 ('if', 184),
 ('any', 182),
 ('or', 181),
 ('so', 181),
 ('but', 160),
 ('themselves', 157),
 ('out', 157),
 ('no', 152),
 ('after', 152),
 ('same', 148),
 ('because', 147),
 ('mo

In [ ]:
# Ex6: Filter stop words
!wget https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/stop-words.txt

--2023-07-07 12:45:17--  https://raw.githubusercontent.com/deanwampler/spark-scala-tutorial/master/data/stop-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3525 (3.4K) [text/plain]
Saving to: ‘stop-words.txt’

stop-words.txt      100%[===================>]   3.44K  --.-KB/s    in 0s      

2023-07-07 12:45:17 (44.5 MB/s) - ‘stop-words.txt’ saved [3525/3525]



In [ ]:
text_words_persisted.take(10)

[('the', 6956),
 ('of', 3192),
 ('and', 3079),
 ('to', 2771),
 ('that', 1485),
 ('in', 1412),
 ('he', 1345),
 ('a', 1273),
 ('they', 1137),
 ('their', 1105)]

In [ ]:
stop_words = sc.textFile("/content/stop-words.txt")
stop_words_list = stop_words.collect()

In [ ]:
stop_words_list.append("a")

In [ ]:
text_words_filtered = text_words_persisted.filter(lambda x: x[0] not in stop_words_list)

In [ ]:
text_words_filtered.take(10)

[('Chapter', 402),
 ('great', 290),
 ('Caesar', 250),
 ('The', 220),
 ('men', 188),
 ('enemy', 186),
 ('Caesar,', 143),
 ('number', 143),
 ('them,', 136),
 ('He', 133)]

In [ ]:
# Stopping Spark-Session and Spark context
sc.stop()
spark.stop()

In [ ]:
df = spark.read.json("/content/sample_data/anscombe.json")

In [ ]:
df.filter(df.Y>4).show(10)

+------+----+-----+---------------+
|Series|   X|    Y|_corrupt_record|
+------+----+-----+---------------+
|     I|10.0| 8.04|           null|
|     I| 8.0| 6.95|           null|
|     I|13.0| 7.58|           null|
|     I| 9.0| 8.81|           null|
|     I|11.0| 8.33|           null|
|     I|14.0| 9.96|           null|
|     I| 6.0| 7.24|           null|
|     I| 4.0| 4.26|           null|
|     I|12.0|10.84|           null|
|     I| 7.0| 4.81|           null|
+------+----+-----+---------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import col
df.where("Series is null").show(10)

+------+----+-----+---------------+
|Series|   X|    Y|_corrupt_record|
+------+----+-----+---------------+
|     I|10.0| 8.04|           null|
|     I| 8.0| 6.95|           null|
|     I|13.0| 7.58|           null|
|     I| 9.0| 8.81|           null|
|     I|11.0| 8.33|           null|
|     I|14.0| 9.96|           null|
|     I| 6.0| 7.24|           null|
|     I| 4.0| 4.26|           null|
|     I|12.0|10.84|           null|
|     I| 7.0| 4.81|           null|
+------+----+-----+---------------+
only showing top 10 rows



In [ ]:
df.createTempView("results")

AttributeError: ignored

In [ ]:
df_res.withColumn

DataFrame[Series: string, X: double, Y: double, _corrupt_record: string]

In [ ]:
gallic = spark.read.text("gallic.mb.txt")

In [ ]:
from pyspark.sql.functions import split
words = gallic.withColumn("words", split("value"," "))

In [ ]:
words.show(10)

+--------------------+--------------------+
|               value|               words|
+--------------------+--------------------+
|                    |                  []|
|              BOOK 1|           [BOOK, 1]|
|                    |                  []|
|           Chapter 1|        [Chapter, 1]|
|                    |                  []|
|All Gaul is divid...|[All, Gaul, is, d...|
|the Aquitani anot...|[the, Aquitani, a...|
|in our Gauls, the...|[in, our, Gauls,,...|
|customs and laws....|[customs, and, la...|
|the Marne and the...|[the, Marne, and,...|
+--------------------+--------------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import explode
word_list = words.select(explode("words").alias("word"))
word_list.show(10)

+-------+
|   word|
+-------+
|       |
|   BOOK|
|      1|
|       |
|Chapter|
|      1|
|       |
|    All|
|   Gaul|
|     is|
+-------+
only showing top 10 rows



In [ ]:
word_count = word_list.groupBy("word").count()

In [ ]:
word_count.show(10)

+-----------+-----+
|       word|count|
+-----------+-----+
|      those|  245|
|       hope|   35|
|        few|   55|
|       some|  106|
| lieutenant|    7|
|    persist|    1|
|         By|   16|
|unequivocal|    1|
|    implore|    2|
|     youth,|    2|
+-----------+-----+
only showing top 10 rows



In [ ]:
word_count.orderBy("count", ascending=False).show(10)

+-----+-----+
| word|count|
+-----+-----+
|  the| 6956|
|   of| 3192|
|  and| 3079|
|   to| 2771|
| that| 1485|
|   in| 1412|
|   he| 1345|
|    a| 1273|
| they| 1137|
|their| 1105|
+-----+-----+
only showing top 10 rows



In [ ]:
stop_words = spark.read.text("stop-words.txt")

In [ ]:
stop_words.show()

+-----------+
|      value|
+-----------+
|       able|
|      about|
|      above|
|  according|
|accordingly|
|     across|
|   actually|
|      after|
| afterwards|
|      again|
|    against|
|      ain't|
|        all|
|      allow|
|     allows|
|     almost|
|      alone|
|      along|
|    already|
|       also|
+-----------+
only showing top 20 rows



In [58]:
from pyspark.sql.functions import broadcast
df_filtered = word_count.join(broadcast(stop_words),word_count.word == stop_words.value,"outer")

In [59]:
df_results = df_filtered.where("value is null").orderBy("count",ascending=False).drop("value")

In [60]:
df_results.show(10)

+-------+-----+
|   word|count|
+-------+-----+
|      a| 1273|
|       |  820|
|Chapter|  402|
|  great|  290|
| Caesar|  250|
|    The|  220|
|    men|  188|
|  enemy|  186|
|Caesar,|  143|
| number|  143|
+-------+-----+
only showing top 10 rows

